In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from random import randint


1 subject.
No feature extraction.

In [4]:
file = "s01.dat"
subject_id = 1
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
    dictRaw = pd.read_pickle(pickle_file) 
    labels = dictRaw.get('labels')
    data = dictRaw.get('data')
                
    dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

    a,b,c = data.shape
    E = data.reshape(40, 322560)
    allData = pd.DataFrame(E).transpose()
    rows = list()
    for _,row in dfLabels.iterrows():
        rows += [row]*8064
    aux = pd.DataFrame(rows).reset_index(drop=True)
    data = allData.merge(aux, left_index=True, right_index=True)

    data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

    data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

    data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

    copia = data.copy()

    y = copia[['valence']]

    x = copia.drop(columns=['valence'])

    xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s02.dat"
subject_id = 2
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
    dictRaw = pd.read_pickle(pickle_file) 
    labels = dictRaw.get('labels')
    data = dictRaw.get('data')
                
    dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

    a,b,c = data.shape
    E = data.reshape(40, 322560)
    allData = pd.DataFrame(E).transpose()
    rows = list()
    for _,row in dfLabels.iterrows():
        rows += [row]*8064
    aux = pd.DataFrame(rows).reset_index(drop=True)
    data = allData.merge(aux, left_index=True, right_index=True)

    data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

    data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

    data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

    copia = data.copy()

    y = copia[['valence']]

    x = copia.drop(columns=['valence'])

    xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s03.dat"
subject_id = 3
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s04.dat"
subject_id = 4
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s05.dat"
subject_id = 5
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s06.dat"
subject_id = 6
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s07.dat"
subject_id = 7
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s08.dat"
subject_id = 8
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s09.dat"
subject_id = 9
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s10.dat"
subject_id = 10
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s11.dat"
subject_id = 11
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s12.dat"
subject_id = 12
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s13.dat"
subject_id = 13
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)
file = "s14.dat"
subject_id = 14
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s15.dat"
subject_id = 15
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s16.dat"
subject_id = 16
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s17.dat"
subject_id = 17
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s18.dat"
subject_id = 18
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s19.dat"
subject_id = 19
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s20.dat"
subject_id = 20
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s21.dat"
subject_id = 21
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s22.dat"
subject_id = 22
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s23.dat"
subject_id = 23
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s24.dat"
subject_id = 24
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s25.dat"
subject_id = 25
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s26.dat"
subject_id = 26
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s27.dat"
subject_id = 27
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s28.dat"
subject_id = 28
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s29.dat"
subject_id = 29
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s30.dat"
subject_id = 30
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

file = "s31.dat"
subject_id = 31
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)
file = "s32.dat"
subject_id = 32
with open(os.path.join('data_preprocessed_python\\',file), 'rb') as pickle_file:
        dictRaw = pd.read_pickle(pickle_file) 
        labels = dictRaw.get('labels')
        data = dictRaw.get('data')
                
        dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

        a,b,c = data.shape
        E = data.reshape(40, 322560)
        allData = pd.DataFrame(E).transpose()
        rows = list()
        for _,row in dfLabels.iterrows():
            rows += [row]*8064
        aux = pd.DataFrame(rows).reset_index(drop=True)
        data = allData.merge(aux, left_index=True, right_index=True)

        data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

        data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
        #data.drop(['arousal', 'dominance', 'liking'], axis=1)

        data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

        copia = data.copy()

        y = copia[['valence']]

        x = copia.drop(columns=['valence'])

        xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)                

param_distribs_rf = {
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5, 10]
}

param_distribs_svm = {
    'kernel': ('linear','rbf'),
    'C': [0.1, 1, 10, 100]
}

param_distribs_knn = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform', 'distance']
}

clf = RandomForestClassifier(random_state=0, verbose=1, n_jobs=5)
rnd_rf_search = GridSearchCV(clf, param_grid=param_distribs_rf, scoring='accuracy') 
rnd_rf_search.fit(xTrain, np.ravel(yTrain))
model = rnd_rf_search.best_estimator_
predict = model.predict(xTest)

from sklearn.metrics import accuracy_score

    
from sklearn import svm
from sklearn.svm import SVC

svm_clf = svm.SVC()
rnd_svm_search = GridSearchCV(svm_clf, param_grid=param_distribs_svm, scoring='accuracy')
rnd_svm_search.fit(xTrain, np.ravel(yTrain))
model = rnd_svm_search.best_estimator_
svm_predict = model.predict(xTest)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
rnd_knn_search = GridSearchCV(knn, param_grid=param_distribs_knn, scoring='accuracy')
rnd_knn_search.fit(xTrain, np.ravel(yTrain))
model = rnd_knn_search.best_estimator_
knn_predict = model.predict(xTest)

params_dict = rnd_rf_search.best_params_
params_dict.update(rnd_svm_search.best_params_)
params_dict.update(rnd_knn_search.best_params_)
with open('experiment1_1_params.csv', 'a') as f: 
    w = csv.DictWriter(f, params_dict.keys())
    w.writerow(params_dict)


from sklearn.metrics import precision_score

rf_prec_happy = precision_score(yTest, predict, pos_label="happy")
rf_prec_sad = precision_score(yTest, predict, pos_label="sad")
svm_prec_happy = precision_score(yTest, svm_predict, pos_label="happy")
svm_prec_sad = precision_score(yTest, svm_predict, pos_label="sad")
knn_prec_happy = precision_score(yTest, knn_predict, pos_label="happy")
knn_prec_sad = precision_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import recall_score

rf_rec_happy = recall_score(yTest, predict, pos_label="happy")
rf_rec_sad = recall_score(yTest, predict, pos_label="sad")
svm_rec_happy = recall_score(yTest, svm_predict, pos_label="happy")
svm_rec_sad = recall_score(yTest, svm_predict, pos_label="sad")
knn_rec_happy = recall_score(yTest, knn_predict, pos_label="happy")
knn_rec_sad = recall_score(yTest, knn_predict, pos_label="sad")

from sklearn.metrics import f1_score

rf_f1_happy = f1_score(yTest, predict, pos_label="happy")
rf_f1_sad = f1_score(yTest, predict, pos_label="sad")
svm_f1_happy = f1_score(yTest, svm_predict, pos_label="happy")
svm_f1_sad  = f1_score(yTest, svm_predict, pos_label="sad")
knn_f1_happy = f1_score(yTest, knn_predict, pos_label="happy")
knn_f1_sad = f1_score(yTest, knn_predict, pos_label="sad")
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy',
             'Happy precision',
             'Sad precision',
             'Happy recall',
             'Sad recall',
             'Happy f1-score',
             'Sad f1-score']

data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc, rf_prec_happy, rf_prec_sad, rf_rec_happy, rf_rec_sad, rf_f1_happy, rf_f1_sad], [subject_id, 1, "-", "-", "-", "SVM", svm_acc, svm_prec_happy, svm_prec_sad, svm_rec_happy, svm_rec_sad, svm_f1_happy, svm_f1_sad], [subject_id, 1, "-", "-", "-", "kNN", knn_acc, knn_prec_happy, knn_prec_sad, knn_rec_happy, knn_rec_sad, knn_f1_happy, knn_f1_sad]
all_data = data_CSV

csv_df = pd.DataFrame(all_data, columns=col_names)

csv_df.to_csv("experiment1_1.csv", mode="a", header=False, index=False)

IndentationError: unexpected indent (<ipython-input-4-cca8cee3d3c2>, line 130)